<a href="https://colab.research.google.com/github/ELehmann91/NLP/blob/master/policy_qa_haystack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Make sure you have a GPU running
!nvidia-smi

Fri Sep 23 08:53:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install pymilvus==1.1.

In [ ]:
!pip install 'farm-haystack[ocr]'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%%capture
# Install the latest release of Haystack in your own environment 
! pip install farm-haystack

# Install the latest master of Haystack
#!pip install grpcio-tools==1.34.1
#!pip install git+https://github.com/deepset-ai/haystack.git

# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.gitegg=farm-haystack[colab]
# If you run this notebook on Google Colab, you might need to
# restart the runtime after installing haystack.

In [ ]:
!wget --no-check-certificate https://dl.xpdfreader.com/xpdf-tools-linux-4.03.tar.gz && tar -xvf xpdf-tools-linux-4.03.tar.gz && sudo cp xpdf-tools-linux-4.03/bin64/pdftotext /usr/local/bin

--2022-09-23 12:15:49--  https://dl.xpdfreader.com/xpdf-tools-linux-4.03.tar.gz
Resolving dl.xpdfreader.com (dl.xpdfreader.com)... 45.79.72.155
Connecting to dl.xpdfreader.com (dl.xpdfreader.com)|45.79.72.155|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-09-23 12:15:50 ERROR 404: Not Found.



In [ ]:
!pip install --upgrade "urllib3==1.25.4" awscli
!pip install pdf2image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
exit(00)

In [ ]:
!pip install 'protobuf == 3.20.1'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.6
    Uninstalling protobuf-4.21.6:
      Successfully uninstalled protobuf-4.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.1 which is incompatible.
grpcio-tools 1.49.1 requires protobuf<5.0dev,>=4.21.3, but you have protobuf 3.20.1 which is incompatible.


# Preprocessing and indexing documents
The report you will be working with is in the form of a PDF file, so start by creating a PDFToTextConverter. This node extracts the textual information from the PDF and saves it into a Python list. 

Set the remove_numeric_tables parameter to True. This is fine to do since the report does not contain many tables. Where this is not the case, you may want to use Haystack’s newly implemented AzureConverter, which is optimized for working with tabular data.

In [ ]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]
pip install pdf2image
pip install pytesseract

In [ ]:
%%bash
wget --no-check-certificate https://dl.xpdfreader.com/xpdf-tools-linux-4.04.tar.gz &&
tar -xvf xpdf-tools-linux-4.04.tar.gz && sudo cp xpdf-tools-linux-4.04/bin64/pdftotext /usr/local/bin

In [ ]:
from haystack.utils import clean_wiki_text,  fetch_archive_from_http, print_answers #convert_files_to_dicts
from haystack.nodes import FARMReader, TransformersReader

In [ ]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [ ]:
!wget https://www.dffe.gov.za/sites/default/files/docs/southafricasINDCupdated2021sept.pdf data/southafricasINDCupdated2021sept.pdf

--2022-09-23 12:19:57--  https://www.dffe.gov.za/sites/default/files/docs/southafricasINDCupdated2021sept.pdf
Resolving www.dffe.gov.za (www.dffe.gov.za)... 164.151.133.35
Connecting to www.dffe.gov.za (www.dffe.gov.za)|164.151.133.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1364580 (1.3M) [application/pdf]
Saving to: ‘southafricasINDCupdated2021sept.pdf.4’

southafricasINDCupd 100%[===================>]   1.30M   867KB/s    in 1.5s    

2022-09-23 12:20:01 (867 KB/s) - ‘southafricasINDCupdated2021sept.pdf.4’ saved [1364580/1364580]

--2022-09-23 12:20:01--  http://data/southafricasINDCupdated2021sept.pdf
Resolving data (data)... failed: No address associated with hostname.
wget: unable to resolve host address ‘data’
FINISHED --2022-09-23 12:20:01--
Total wall clock time: 3.1s
Downloaded: 1 files, 1.3M in 1.5s (867 KB/s)


In [ ]:
from haystack.nodes import PDFToTextConverter

pdf_converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=['de','en'])

converted = pdf_converter.convert(file_path = 'southafricasINDCupdated2021sept.pdf', meta = { 'document': 'NDC', 'processed': False })

Note that you’ll be adding metadata to the converted documents. Once you index this document (as well as all subsequent documents) into a database, the document’s metadata will allow you to filter them. For instance, by filtering based on the ‘company’ metadata, you can specify that you only want to ask a question applicable only to documents concerning ‘Company_1’ and ‘Company_3’ rather than all documents in the database.

Now you’ll build a PreProcessor to split your documents into chunks of length 200 and an overlap of ten words.

In [ ]:
from haystack.nodes import PreProcessor

preprocessor = PreProcessor(clean_empty_lines=True,
                            clean_whitespace=True,
                            clean_header_footer=True,
                            split_by='sentence',
                            split_length=4,
                            split_respect_sentence_boundary=False,
                            split_overlap=1)

preprocessed = preprocessor.process(converted)

Preprocessing:   0%|          | 0/1 [00:00<?, ?docs/s]

In [ ]:
preprocessed[1]

<Document: {'content': "Mitigation component of the NDC ........................................................................................................................ 14\n4 (a) Approach to setting updated NDC target ranges ................................................................................................. 14\n4 (b) Methodological consistency in setting and accounting for NDC targets .............................................................. 14\n4 (c) Mitigation targets for 2025 and 2030 .................................................................................................................. 15\n4 (d) Information to facilitate clarity, transparency and understanding of mitigation ................................................ 16\n5. Support requirements under the Convention and Paris Agreement ...................................................................... 28\n6. Equitable access to sustainable development ...................................

With this out of the way, you can now index your documents into an Elasticsearch database.

In [ ]:
# In Colab / No Docker environments: Start Elasticsearch from source
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30


In [ ]:
from haystack.document_stores import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore()
document_store.delete_all_documents()
document_store.write_documents(preprocessed)

                1. delete_all_documents() method is deprecated, please use delete_documents method
                For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/1045
                


#Building a Question Answering Pipeline

Question answering pipelines consist of two main components — the Retriever and the Reader. You can initialize these with only a few lines of code:

In [ ]:
from haystack.nodes import DensePassageRetriever, FARMReader

retriever = DensePassageRetriever(document_store=document_store)

reader = FARMReader(model_name_or_path='deepset/roberta-base-squad2', use_gpu=True)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


Once initialized, you will also need to update your database with [dense vector embeddings](https://www.deepset.ai/blog/what-is-text-vectorization-in-nlp). This is as simple as executing a single line of code:

In [ ]:
document_store.update_embeddings(retriever)

Updating embeddings:   0%|          | 0/179 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/192 [00:00<?, ? Docs/s]

With both your retriever and reader in place, you can place them into a Pipeline.

In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipeline = ExtractiveQAPipeline(reader, retriever)

#Information Extraction with Question Answering
Before you can use our pipeline, you’ll need to come up with some questions. Let’s create a few below:

In [ ]:
questions = [ 'Was ist das Ziel des Projekts?',
              'In welchem Land wird das Projekt umgesetzt?',
              'Zu welchem Zeitpunkt  startet das Projekt?',
              'Zu welchem Zeitpunkt ist das Projekt abgeschlossen?',
              'Welche Sektoren sind betroffen?',
              'Wie hoch ist das Funding des Projekts?',
              'Gibt es Risiken in der Umsetzung?' ]

In [ ]:
questions = [ 'What is the goal of the project?',
              'Which countries are involved?',
              'what is the time horizon?',
              'which sectors are mentioned?' ]

In [ ]:
questions = [ 'What are the goals and efforts to mitigate climate change?',
              'What are the goals and efforts to adapt to climate change?',
              'Which policies are mentioned?',
              'Is a certain budget mentioned?' ]

Next, iterate over the questions and feed them into your pipeline.

Set the top_k parameters to 50 and 1 for the retriever and the reader, respectively. A top_k value of 50 for retriever is comparatively high and may slow down a question answering system with many active users. For a QA system in production, the higher speed achieved by decreasing the top_k parameter is generally worth a small loss in accuracy. However, because right now your information extraction system probably doesn’t have to serve hundreds of users and you’ll want the returned results to be as accurate as possible, you’re better served by amping the top_k value up. As for the reader, you’ll only be interested in the single highest-ranking answer, so set its top_k value to 1.

In [ ]:
answers = []

for question in questions:
    prediction = pipeline.run(query=question,
                 params = {'Retriever': {'top_k': 5},
                           'Reader': { 'top_k': 1 } })
    answers.append(prediction)


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 15.80 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 41.47 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 57.79 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 63.75 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 46.69 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 22.67 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 39.96 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 23.93 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 43.76 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 40.01 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  9.49 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 23.98 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 41.82 Batches/s]

Inferencing Samples: 100%|██████████|

For each question that you’ll pass through the pipeline, it returns a dictionary that contains both the query and the answer. The pipeline saved each of these dictionaries into a list, so to print out the results, simply iterate over the list and output the dictionary’s values by accessing them via their corresponding keys.

In [ ]:
for answer in answers:
    print('Q:', answer['query'])
    print('A:', answer['answers'][0].answer)
    print('A:', answer['answers'][0].context)
    print('A:', answer['answers'][0].document_id)
    print('\n')


Q: What is the goal of the project?
A: Goal 5: Quantification and
acknowledgement of the
national adaptation and
resilience efforts
A: g
countries for
recognition. Goal 5: Quantification and
acknowledgement of the
national adaptation and
resilience efforts. The adaptation efforts have
A: 6b2c27fde511b87e14eced9245cda66d


Q: Which countries are involved?
A: Africa and South Africa
A: lobal nature of the adaptation responsibility. In
this respect, Africa and South Africa, are warming at a rate that is above twice the global average

A: 18e21ce054cea90bc8e4779fda993098


Q: what is the time horizon?
A: two five-year
A: These targets for 2025 and 2030 correspond to two five-year time frames, and
corresponding periods of implementation, from 1 January 2021 to 31 Decemb
A: 5ddfc6bf98442559f9b88d6fe81d98ec


Q: which sectors are mentioned?
A: health, water, biodiversity, agriculture and
human settlements
A:  sectors recommended for prioritisation
are health, water, biodiversity, agriculture 

# Summarizer

In [ ]:
from haystack.nodes import TransformersSummarizer
from haystack import Document


In [ ]:
summarizer = TransformersSummarizer(model_name_or_path="sshleifer/distilbart-cnn-12-6") #ml6team/mt5-small-german-finetune-mlsum


In [ ]:
preprocessed[25].content

'Further details on sectoral policies and measures are reported in South Africa’s Fourth Biennial Update\nReport, summarised in its Table ES-3. By implementing these domestic measures, South Africa is\nshowing that implementation is part of our ambition. 5\nThe long-term decarbonization of the South African economy will in the 2020s focus primarily on the\nelectricity sector; in the 2030s, a deeper transition will take place in the electricity sector, coupled with\na transition in the transport sector towards low emission vehicles; while the 2040s and beyond will be\ncharacterized by the decarbonization of the hard-to-mitigate sectors. The key challenge during the\nimplementation periods of this first NDC (2021 to 2025, and 2026 to 2030) will be the transition in the\nelectricity sector, seeking early investment in and preparing for mitigation in harder-to-mitigate\nsectors, and addressing the economic and social consequences resulting from this transition in coal-\nproducing areas.'

In [ ]:
summary = summarizer.predict(documents=preprocessed[25], generate_single_summary=True)

In [ ]:
summary[0].content

' South Africa’s first Adaptation Communication, serving as the Adaptation component of the NDC. South Africa updates and enhances its nationally determined contribution under the Paris Agreement, meeting its obligation under Article 4.9 to communicate NDCs every five years.'

# Classifier

In [ ]:
from haystack.nodes import TransformersDocumentClassifier


doc_classifier = TransformersDocumentClassifier(
    model_name_or_path="cross-encoder/nli-distilroberta-base",
    task="zero-shot-classification",
    labels=["transport", "carbon","agriculture","disaster risk"],
    batch_size=16,
)

In [ ]:
classified_docs = doc_classifier.predict(preprocessed)


Classifying documents:   0%|          | 0/179 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1039: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


In [ ]:
# let's see how it looks: there should be a classification result in the meta entry containing labels and scores.
print(classified_docs[25].to_dict())


{'content': 'Further details on sectoral policies and measures are reported in South Africa’s Fourth Biennial Update\nReport, summarised in its Table ES-3. By implementing these domestic measures, South Africa is\nshowing that implementation is part of our ambition. 5\nThe long-term decarbonization of the South African economy will in the 2020s focus primarily on the\nelectricity sector; in the 2030s, a deeper transition will take place in the electricity sector, coupled with\na transition in the transport sector towards low emission vehicles; while the 2040s and beyond will be\ncharacterized by the decarbonization of the hard-to-mitigate sectors. The key challenge during the\nimplementation periods of this first NDC (2021 to 2025, and 2026 to 2030) will be the transition in the\nelectricity sector, seeking early investment in and preparing for mitigation in harder-to-mitigate\nsectors, and addressing the economic and social consequences resulting from this transition in coal-\nproduci

In [ ]:
classified_docs[25]

<Document: {'content': 'Further details on sectoral policies and measures are reported in South Africa’s Fourth Biennial Update\nReport, summarised in its Table ES-3. By implementing these domestic measures, South Africa is\nshowing that implementation is part of our ambition. 5\nThe long-term decarbonization of the South African economy will in the 2020s focus primarily on the\nelectricity sector; in the 2030s, a deeper transition will take place in the electricity sector, coupled with\na transition in the transport sector towards low emission vehicles; while the 2040s and beyond will be\ncharacterized by the decarbonization of the hard-to-mitigate sectors. The key challenge during the\nimplementation periods of this first NDC (2021 to 2025, and 2026 to 2030) will be the transition in the\nelectricity sector, seeking early investment in and preparing for mitigation in harder-to-mitigate\nsectors, and addressing the economic and social consequences resulting from this transition in coa

# Entity

In [ ]:
from haystack.nodes import EntityExtractor

entity_extractor = EntityExtractor(model_name_or_path="fhswf/bert_de_ner")


In [ ]:
entities = entity_extractor.extract(
    text= preprocessed[1]['content']
 )
entities

In [ ]:
#pipeline = Pipeline()
#pipeline.add_node(text_converter, "TextConverter", ["File"])
#pipeline.add_node(preprocessor, "PreProcessor", ["TextConverter"])
#pipeline.add_node(entity_extractor, "EntityExtractor", ["PreProcessor"])
#pipeline.add_node(document_store, "DocumentStore", ["EntityExtractor"])

#pipeline.run(file_paths=file_paths)


#Crawl

In [ ]:
!mkdir 'crawled_files_giz'

In [ ]:
%%capture
!pip install webdriver_manager
!pip install selenium
!apt-get update
!apt install chromium-driver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
from haystack.nodes import Crawler

crawler = Crawler(output_dir="crawled_files_giz")
docs = crawler.crawl(
    urls=["https://www.giz.de/de/weltweit/324.html"],
    filter_urls=["haystack"],
    crawler_depth=1
)